## Create data

In [1]:
import numpy as np

input image

In [58]:
X = [
    [ 
     [2, 1, 0, 0, 2], 
     [1, 1, 0, 1, 1], 
     [1, 2, 2, 1, 0], 
     [0, 0, 2, 0, 0], 
     [1, 0, 1, 0, 1]], 
    [
     [2, 2, 1, 2, 1], 
     [1, 2, 1, 0, 2], 
     [0, 1, 1, 0, 2], 
     [0, 2, 1, 2, 2], 
     [0, 1, 0, 1, 2]], 
    [
     [0, 2, 0, 2, 1], 
     [0, 2, 2, 1, 1], 
     [1, 0, 1, 2, 0], 
     [1, 0, 1, 2, 0], 
     [0, 0, 0, 0, 0]]
]

In [59]:
X = np.array([X])
X.shape

(1, 3, 5, 5)

filters

In [60]:
W0 = [
    [[1, 0, -1], 
     [1, 1, -1], 
     [0, 0, -1]], 
    [[0, 0, 1], 
     [1, 1, 0], 
     [0, 0, 0]], 
    [[-1, 1, 0], 
     [0, 0, 0], 
     [0, -1, 0]]
]
W0 = np.array(W0)
W0.shape

(3, 3, 3)

In [61]:
W1 = [
    [[-1, 0, -1], 
     [1, 0, 0], 
     [1, 0, 1]], 
    [[-1, 1, 0], 
     [1, 1, 0], 
     [-1, 0, 1]], 
    [[-1, 1, 1], 
     [-1, -1, 0], 
     [0, 1, 1]]
]
W1 = np.array(W1)
W1.shape

(3, 3, 3)

In [108]:
W = [list(W0), list(W1)]
W = np.array(W)
W.shape

(2, 3, 3, 3)

## Hyper parameters

In [106]:
S = 2 # stride
P = 1 # zero padding
W_size = 5 # input size
K = 2 # number of filters
F_size = 3 # filter size

In [107]:
# Bias
B1 = 1
B2 = 0

Calculate output shape

In [114]:
n_filters, d_filter, h_filter, w_filter = W.shape
n_x, d_x, h_x, w_x = X.shape

h_out = (h_x - h_filter + 2 * P) / S + 1
w_out = (w_x - w_filter + 2 * P) / S + 1
h_out, w_out = int(h_out), int(w_out)

h_out, w_out

(3, 3)

In [ ]:
def get_im2col_indices(x_shape, field_height, field_width, padding=1, stride=1):
  # First figure out what the size of the output should be
  N, C, H, W = x_shape
  assert (H + 2 * padding - field_height) % stride == 0
  assert (W + 2 * padding - field_height) % stride == 0
  out_height = (H + 2 * padding - field_height) / stride + 1
  out_width = (W + 2 * padding - field_width) / stride + 1
  
  out_height, out_width = int(out_height), int(out_width)
  i0 = np.repeat(np.arange(field_height), field_width)
  i0 = np.tile(i0, C)
  i1 = stride * np.repeat(np.arange(out_height), out_width)
  j0 = np.tile(np.arange(field_width), field_height * C)
  
  j1 = stride * np.tile(np.arange(out_width), out_height)
  i = i0.reshape(-1, 1) + i1.reshape(1, -1)
  j = j0.reshape(-1, 1) + j1.reshape(1, -1)

  k = np.repeat(np.arange(C), field_height * field_width).reshape(-1, 1)

  return (k, i, j)

def im2col_indices(x, field_height, field_width, padding=1, stride=1):
  """ An implementation of im2col based on some fancy indexing """
  # Zero-pad the input
  p = padding
  x_padded = np.pad(x, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')

  k, i, j = get_im2col_indices(x.shape, field_height, field_width, padding,
                               stride)

  cols = x_padded[:, k, i, j]
  C = x.shape[1]
  cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
  return cols

Reshape X and W to apply convolution without using loops

In [111]:
X_col = im2col_indices(X, 3, 3, P, S)
X_col.shape

(27, 9)

In [122]:
X_col

array([[0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 2, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 2, 1, 0, 0, 0],
       [2, 0, 2, 1, 2, 0, 1, 1, 1],
       [1, 0, 0, 2, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 2, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 2, 2],
       [0, 0, 0, 1, 1, 2, 0, 1, 2],
       [0, 0, 0, 2, 0, 0, 2, 2, 0],
       [0, 2, 2, 0, 1, 0, 0, 1, 1],
       [2, 1, 1, 0, 1, 2, 0, 0, 2],
       [2, 2, 0, 1, 0, 0, 1, 1, 0],
       [0, 2, 0, 0, 2, 2, 0, 0, 0],
       [1, 1, 2, 0, 1, 2, 0, 0, 0],
       [2, 0, 0, 2, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 1, 0, 0, 2],
       [0, 0, 0, 0, 2, 1, 1, 1, 0],
       [0, 0, 0, 2, 1, 0, 0, 2, 0],
       [0, 2, 2, 0, 0, 2, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 0],
       [2, 2, 0, 0, 2, 0, 0, 0, 0],
       [0, 2, 1, 0, 0, 2, 0, 0, 0],
       [0, 2, 1, 1, 1, 0, 0, 0, 0],
       [2, 1, 0, 0, 2, 0, 0, 0, 0]])

In [121]:
W_col = W.reshape(K, -1)
W_col, W_col.shape

(array([[ 1,  0, -1,  1,  1, -1,  0,  0, -1,  0,  0,  1,  1,  1,  0,  0,  0,
          0, -1,  1,  0,  0,  0,  0,  0, -1,  0],
        [-1,  0, -1,  1,  0,  0,  1,  0,  1, -1,  1,  0,  1,  1,  0, -1,  0,
          1, -1,  1,  1, -1, -1,  0,  0,  1,  1]]), (2, 27))

### Convolution output

In [113]:
out = W_col @ X_col
out

array([[ 2,  1,  4, -1,  4,  4,  4,  5,  2],
       [ 7,  5,  2,  4,  4,  0,  1,  3,  1]])

In [117]:
out = W_col @ X_col
out = out.reshape(n_x, n_filters, h_out, w_out)
out

array([[[[ 2,  1,  4],
         [-1,  4,  4],
         [ 4,  5,  2]],

        [[ 7,  5,  2],
         [ 4,  4,  0],
         [ 1,  3,  1]]]])